In [25]:
# Import libraries
import pygame
import random

In [26]:
class Scenario:

    def __init__(self, x_position, y_position, face, speed):
        self.x_position = x_position
        self.y_position = y_position
        self.face = face
        self.speed = speed
        self.rect = self.face.get_rect()
        self.rect.x = self.x_position
        self.rect.y = self.y_position
      
    def __repr__(self):
        return self.face

class Guy(Scenario):
    
    def __init__(self, x_position, y_positionface, face, speed, is_live, has_alcohol=False):
        super().__init__(x_position, y_positionface, face, speed)
        self.is_live = is_live
        self.has_alcohol = has_alcohol
        
class House(Scenario):
    
    def __init__(self, x_position, y_positionface, face, speed, alcohol_total=0):
        super().__init__(x_position, y_positionface, face, speed)
        self.alcohol_total = alcohol_total

In [27]:
# Initialize all imported pygame module
pygame.init()

# Set the screen with 900x500 pixels
WIDTH = 900
HEIGHT = 500
screen = pygame.display.set_mode((WIDTH, HEIGHT))

# Set difficulty level
n_virus = 4
virus_max_speed = 10
guy_speed = 3

# Final goal
n_alcohol = 3

# Create scenario items 
background = Scenario(0, 0, pygame.image.load('background.bmp'), 0)
house = House(random.randrange(0, WIDTH - 100), random.randrange(0, HEIGHT - 100), pygame.image.load('house.png'), 0)
guy = Guy(house.x_position + 20, house.y_position + 20, pygame.image.load('guy.png'), guy_speed, True)
alcohol = Scenario(random.randrange(0,  WIDTH - 100), random.randrange(0, HEIGHT - 100), pygame.image.load('alcohol.png'), 0)
sick = Scenario(200, 0, pygame.image.load('sick.png'), 0)

# Declare the variables to signal if the player stops to press the keyboard
fl_moving_right, fl_moving_left = False, False
fl_moving_up, fl_moving_down = False, False

def virus_factory():
    # This function creates a list of objects for the viruses
    viruses = []
    
    # The range is the total of virus that the game has. In this case we create 5 viruses with speed between 1 to 4
    for i in range(n_virus):
        viruses.append(Scenario(0, -30, pygame.image.load('virus.png'), random.randrange(1, virus_max_speed)))
    
    return viruses
        
viruses = virus_factory()

def virus_behavior():
    """
    This function moves the virus through the screen and checks if the virus hits the guy outside house
    """
    for v in viruses:
        v.y_position += v.speed
        
        # If the virus exceed the limit returns to the top in a different position
        if v.y_position > HEIGHT:
            v.x_position = random.randrange(0, WIDTH)
            v.y_position = -30
        
        # Update the rect to check the collision
        v.rect.x = v.x_position
        v.rect.y = v.y_position

        screen.blit(v.face, (v.x_position, v.y_position))
        
        # If the virus collides with the guy, display the sick emoji and stop the game
        if v.rect.colliderect(guy.rect) and not house.rect.contains(guy.rect):
            screen.blit(sick.face, (sick.x_position, sick.y_position))
            guy.is_live = False

def has_acohol():
    """
    This function updates the guy's image when it gets the alcohol
    """
    if guy.rect.colliderect(alcohol.rect):
        guy.face = pygame.image.load('guy_alc.png')
        guy.has_alcohol = True


def set_scenario():
    """
    Create the scenario and start the virus behavior
    """
    screen.blit(background.face, (background.x_position, background.y_position))
    screen.blit(guy.face, (guy.x_position, guy.y_position))
    screen.blit(house.face, (house.x_position, house.y_position))
    screen.blit(alcohol.face, (alcohol.x_position, alcohol.y_position))
    virus_behavior()
    has_acohol()


def guy_behavior():
    """
    This function gets the keyboard input and it moves the guy to right, left, up or down
    """
    global fl_moving_right, fl_moving_left, fl_moving_up, fl_moving_down

    for event in pygame.event.get():
            if event.type == pygame.KEYDOWN and event.key == pygame.K_RIGHT:
                fl_moving_right = True
            if event.type == pygame.KEYUP and event.key == pygame.K_RIGHT:
                fl_moving_right = False

            if event.type == pygame.KEYDOWN and event.key == pygame.K_LEFT:
                fl_moving_left = True
            if event.type == pygame.KEYUP and event.key == pygame.K_LEFT:
                fl_moving_left = False


            if event.type == pygame.KEYDOWN and event.key == pygame.K_UP:
                fl_moving_up = True
            if event.type == pygame.KEYUP and event.key == pygame.K_UP:
                fl_moving_up = False

            if event.type == pygame.KEYDOWN and event.key == pygame.K_DOWN:
                fl_moving_down = True
            if event.type == pygame.KEYUP and event.key == pygame.K_DOWN:
                fl_moving_down = False

    # Update the guy's position related to the speed inside the limits of screen   
    if fl_moving_right and guy.x_position < WIDTH - 50:
        guy.x_position += guy.speed
    if fl_moving_left and guy.x_position > 0:
        guy.x_position -= guy.speed
    if fl_moving_up and guy.y_position > 0:
        guy.y_position -= guy.speed
    if fl_moving_down and guy.y_position < HEIGHT - 50:
        guy.y_position += guy.speed

    guy.rect.x = guy.x_position
    guy.rect.y = guy.y_position
    
    # If the guy is inside the house, leave the alcohol and update the house and guy image 
    if house.rect.contains(guy.rect) and guy.has_alcohol and house.alcohol_total <= n_alcohol:
        guy.has_alcohol = False
        house.alcohol_total += 1
        house_face = 'house_' + str(house.alcohol_total) + '.png'
        house.face = pygame.image.load(house_face)
        guy.face = pygame.image.load('guy.png')
                   

try:
    # While the player do not stop the code, keep running
    while True:
        # Do not update the scenario if the virus hits the guy
        if guy.is_live:           
            
            if house.alcohol_total < n_alcohol:
                guy_behavior()
                set_scenario()

                # Update screen
                pygame.display.flip()

except:
    pygame.display.quit()
